# Chromosome Mutation Analysis

### Introduction  
This notebook provides a framework for processing genetic data into a format optimized for visualization. The code below will take a file of somatic genetic mutations as input and output three different files that can be then used to create visualizations in R. 

### Preliminaries

In [10]:
# Import the required packages
import pandas as pd
import numpy as np

### Setting Up Our Data

In [11]:
# Read in the data
somatic = pd.read_csv('somatic_snps.tsv.gz',sep='\t')

In [12]:
# Check out the format of the data
somatic.head()

,CHROM,POS,REF,ALT,A01,A03,A04,A05,A06,B01,...,F06,G02,G03,G04,G05,G06,H02,H04,H05,H06
0,1,13395,T,C,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,15665,G,T,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,48173,C,T,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,52727,C,G,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,55211,C,A,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
chromosomes_in_data = list(somatic['CHROM'].unique()[0:23])

**A Note on Data Structure**  
We need our data frame to be structured in a very specific way for  the rest of the code contained within this notebook to function properly. The main characteristics that we need are as follows:  
* There is a column for the chromosome number
* There is a column for the base pair position
* There is a column for each patient, with each value either being 0 or 1 indicating the presence of a mutation at that location.
* There is a "total" column that indicates the number of mutations across all patients at a given location.

If your data is not in this format the code below will not function. 

**Add Column for Total Mutations**  
Our data matches all of the conditions aside from the last one. We can quickly add a "total" column to meet this condition.

In [14]:
# Add the total column and check to make sure it was added
somatic['total']= somatic.iloc[:, 4:43].sum(axis=1)
somatic.head()

,CHROM,POS,REF,ALT,A01,A03,A04,A05,A06,B01,...,G02,G03,G04,G05,G06,H02,H04,H05,H06,total
0,1,13395,T,C,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,15665,G,T,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,48173,C,T,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,1,52727,C,G,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,1,55211,C,A,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


**Reference Dictionary**  
We need to create a refence dictionary that will contain the lengths of each chromosome. This information will be needed for when we calculate the number of windows to use on a single chromosome. 

In [15]:
chrom_data = {
1:247249719,
2:242951149,
3:199501827,
4:191273063,
5:180857866,
6:170899992,
7:158821424,
8:146274826,
9:140273252,
10:135374737,
11:134452384,
12:132349534,
13:114142980,
14:106368585,
15:100338915,
16:88827254,
17:78774742,
18:76117153,
19:63811651,
20:62435964,
21:46944323,
22:49691432}

### Making Functions

**Function One: Mutation Occurence**  
This first function will calculate "occurence" - the raw number of total mutations within each window on each chromosome. The function will only have one input - window size - and will return a data frame with three columns: (1) chromosome number, (2) window number, and (3) number of mutations found.  
  
*A note on our functions:*  
You will see in the code below that we use the same windows for each chromsome. This is not optimal as it results in us calculating metrics for windows that we know have no data. This will be updated in a futrue iteration, but it is not so detrimental as to noticeably slow the computation time and so we can proceed with this setup for now.  

In [16]:
def occurrence(window_size):
    
    ## Calculate the number of windows we need
    # Because it is the longest, we will use chromosome 1 as the basis for our number of windows
    chrom_len = chrom_data[1]  
    num_windows = int(np.ceil(chrom_len/window_size))
    
    # Create a blank dictionary for saving information
    mutation_dict = {}
        
    # For each chromosome
    for i in chromosomes_in_data:
            
        # Create subset of main data
        subset = 'CHROM=={}'.format(i)
        spec_chrom = somatic.query(subset)
        
        # For each window in the chromosome
        for j in range(1,num_windows):
            # Set the window start and stop positions
            window_start = (j-1)*window_size
            window_stop =  j*window_size
            
            # Slice our df again into a smaller one that only covers the specified window 
            window_df = spec_chrom.query("POS > {} & POS <= {}".format(window_start,window_stop))
            
            # Count the number of mutations within the window
            mutations = window_df['total'].sum()

            # Add to dictionary
            dict_string = '{}_{}'.format(i, j)
            mutation_dict[dict_string] = {'chromosome':i,'window':j,'mutations':mutations}

    # Turn dictionary into dataframe
    df = pd.DataFrame.from_dict(mutation_dict, orient='index')
    
    # Return df
    return df

In [17]:
# Let's try it out! We will use a window size of 1,000,000
trial_occurrence = occurrence(1000000)

In [18]:
# We can take a look at our results to see if it worked
trial_occurrence.head()

,chromosome,window,mutations
1_1,1,1,151
1_2,1,2,232
1_3,1,3,207
1_4,1,4,207
1_5,1,5,152


In [19]:
# Save results to csv
trial_occurrence.to_csv('chromosome_mutations_occurrence.csv')

**Function Two: Mutation Coverage**  
The next thing we want to analyze is coverage - how many patients show a mutation in a given window. This value will fall within the range (0,1) and it will tell us which areas on the chromosome most commonly show mutations across our patient pool. Before we can create this function we need to get a list of the columns within our data frame that correspond to our patients. We will iterate through this list within our function.

In [20]:
# Getting a list of patients
patients = somatic.columns[4:43].to_list()

In [21]:
def coverage(window_size):
    
    ## Calculate the number of windows we need
    # Because it is the longest, we will use chromosome 1 as the basis for our number of windows
    chrom_len = chrom_data[1]
    num_windows = int(np.ceil(chrom_len/window_size))
    
    # Create a blank dictionary for saving information
    density_dict = {}
        
    # For each chromosome
    for i in chromosomes_in_data:
            
        # Create subset of main data
        subset = 'CHROM=={}'.format(i)
        spec_chrom = somatic.query(subset)
        
        # For each window
        for j in range(1,num_windows):
            window_start = (j-1)*window_size
            window_stop =  j*window_size

            window_df = spec_chrom.query("POS > {} & POS <= {}".format(window_start,window_stop))
            
            patient_count = 0

            # For each patient
            for k in patients:
                if window_df[k].sum() > 0:
                    patient_count += 1
           
            # Calculate coverage
            coverage = patient_count / len(patients)                

            # Add to dictionary
            dict_string = '{}_{}'.format(i, j)
            density_dict[dict_string] = {'chromosome':i,'window':j,'coverage':coverage}

    # Turn dictionary into dataframe
    df = pd.DataFrame.from_dict(density_dict, orient='index')
    
    # Return df
    return df

In [22]:
# Let's try it out! We will use a window size of 1,000,000
trial_coverage = coverage(1000000)

In [23]:
# Look at results
trial_coverage.head()

,chromosome,window,coverage
1_1,1,1,0.897436
1_2,1,2,0.974359
1_3,1,3,1.000000
1_4,1,4,0.948718
1_5,1,5,0.897436


In [24]:
# Save results to csv
trial_coverage.to_csv('chromosome_mutations_coverage.csv')

**Function Three: Mutation Density**  
The last thing we want to analyze is density - how close are mutations to one another within each window? There are many ways to think about this, but we will use interquartile range. It is worth noting that this metric, as calculated below, can be a little misleading as it is does not account for the fact that mutations are occuring across different patients. Instead, it analyzes the data as if these mutations were all occuring within a single persons genome. 

In [25]:
def density(window_size):
    
    ## Calculate the number of windows we need
    # Because it is the longest, we will use chromosome 1 as the basis for our number of windows
    chrom_len = chrom_data[1]
    num_windows = int(np.ceil(chrom_len/window_size))
    
    # Create a blank dictionary for saving information
    density_dict = {}
        
    # For each chromosome
    for i in chromosomes_in_data:
            
        # Create subset of main data
        subset = 'CHROM=={}'.format(i)
        spec_chrom = somatic.query(subset)
        
        # For each window
        for j in range(1,num_windows):
            window_start = (j-1)*window_size
            window_stop =  j*window_size

            window_df = spec_chrom.query("POS > {} & POS <= {}".format(window_start,window_stop))
            
            # Get list of mutation positions
            locations = window_df['POS']
            
            # Calculate IQR
            if len(locations) > 0:
                iqr = np.percentile(locations, 75) - np.percentile(locations, 25)
            else:
                iqr = np.nan

            # Add to dictionary
            dict_string = '{}_{}'.format(i, j)
            density_dict[dict_string] = {'chromosome':i,'window':j,'iqr':iqr}

    # Turn dictionary into dataframe
    df = pd.DataFrame.from_dict(density_dict, orient='index')
    
    # Return df
    return df

In [26]:
# Let's try it out! We will use a window size of 1,000,000
trial_density = density(1000000)

In [27]:
# Look at results
trial_density.head()

,chromosome,window,iqr
1_1,1,1,316218.25
1_2,1,2,356773.75
1_3,1,3,332609.00
1_4,1,4,476720.00
1_5,1,5,476802.75


In [28]:
# Save to csv
trial_density.to_csv('chromosome_mutations_density.csv')